# Use examples of [premise](https://github.com/romainsacchi/premise) with user-generated scenarios

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `premise` to adapt the life cycle inventory database [ecoinvent](https://www.ecoinvent.org/) for prospective environmental impact assessment, using **user-generated scenarios**.

User-generated scenario are scenarios built by the premise users community. They can be used on their own
or together with a global IAM scenario.
Public community scenarios are listed under the repository [Premise community scenario](https://github.com/premise-community-scenarios).

User-generated scenarios are "packaged" into [data packages](https://specs.frictionlessdata.io/data-package/).

Data packages ensure that the data used comes with all necessary metadata, resources (scenario data, inventories, etc.), and that the data is formatted in a correct way.

To fetch a data packge, you can use the `datapackage` library.
For exmaple, let's fetch the user-generated scenario about [Switzerland's future energy supply](https://github.com/premise-community-scenarios/energy-perspective-2050-switzerland).

In [3]:
from datapackage import Package

In [4]:
# URL pointing to the raw datapackage.json file of the custom scenario "switzerland_2050"
url = "https://raw.githubusercontent.com/premise-community-scenarios/energy-perspective-2050-switzerland/main/datapackage.json"
switzerland_2050 = Package(url)

#save the name of custom scenario for later automatic db naming
customscenario=switzerland_2050.descriptor['name']
print(customscenario)

energy-perspective-2050-switzerland


In [5]:
#check version of premise. parameter can e sued for db naming
import premise
v=str(premise.__version__).replace(',','.')
for character in ')( ':
    v = v.replace(character, '')
    
pv="PREMISE v"+v
print(pv)

PREMISE v1.3.6


In [6]:
from premise import *
import brightway2 as bw

In [7]:
#restore ecoinvent
if 'ei38-teaching' not in bw.projects:
    bw.restore_project_directory("/srv/data/projects/ecoinvent38.tar.gz")

In [8]:
#bw.projects.copy_project("moeschpond1")

In [9]:
bw.projects.set_current("moeschpond1")

In [10]:
bw.databases

Databases dictionary with 10 object(s):
	EI 3.8 cutoff image SSP2-Base 2050
	EI 3.8 cutoff image SSP2-RCP19 2050
	EI 3.8 cutoff image SSP2-RCP26 2050
	EI 3.8 cutoff remind SSP2-Base 2050
	EI 3.8 cutoff remind SSP2-PkBudg1150 2050
	EI 3.8 cutoff remind SSP2-PkBudg500 2050
	biosphere3
	ei 3.8 cutoff
	super_db_2022-10-26
	super_db_2022-10-27

In [11]:
#define scenarios to be included by premise (https://premise.readthedocs.io/en/latest/extract.html)
scenarios = [
    {"model": "image", "pathway":"SSP2-Base", "year": 2050},
    {"model": "image", "pathway":"SSP2-RCP26", "year": 2050},
    {"model": "image", "pathway":"SSP2-RCP19", "year": 2050},
    {"model": "remind", "pathway":"SSP2-Base", "year": 2050},
    {"model": "remind", "pathway":"SSP2-PkBudg1150", "year": 2050},
    {"model": "remind", "pathway":"SSP2-PkBudg500", "year": 2050},
]

db_original="ei 3.8 cutoff"

In [25]:
%%time



ndb = NewDatabase(
        scenarios = scenarios,        
        source_db=db_original,
        source_version="3.8",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=', #encryption key for the image/remind data
        external_scenarios=[
            switzerland_2050,
        ],
        keep_uncertainty_data=True #keep uncertainty in dataset (not default)
)


premise v.(1, 3, 6)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

To integrate the projections of the user-generated scenario, call `update_external_scenario()`.

In [ ]:
#%%time #should take 10 minutes

#ndb.update_all() #update all sectors that PREMISE can do

ndb.update_electricity() #in this case we are only interested in transforming the electricity sector of the background database

ndb.update_external_scenario()#also customscenarioate the external switzerland2050 scenario

You can combine the user-generated scenario with any, all or none of the projections for the IAM model.
For example, here with the electricity projections of the IMAGE SSP2-Base scenario:

Once the projections are integrated, you can export the database(s) back to your Brightway2 project, to
a CSV Simapro file, or as a set of sparse amtrices (see main exmaples notebook for more details).

In [12]:


#parametric db naming to be used to write db names consistenntly based on defined scenarios
def dbName(scenario_nr:int):
    dbname=db_original
    for a,b in scenarios[scenario_nr].items():
        dbname = dbname + "_"+str(b)
    
    dbname=dbname+"_"+pv
    dbname=dbname+"_"+customscenario
    return dbname

The database name includes relevant metadata: EI version, as well as PREMSIE scenario and version. It does nto include external scenarios

In [14]:
dbName(0)

'ei 3.8 cutoff_image_SSP2-Base_2050_PREMISE v1.3.6_energy-perspective-2050-switzerland'

In [38]:
#superstructure database does only work in activity-browser, but not for brightway 2/2.5 coding
#so we need to export a separate DB for each scenario 

#write the db names to reflect the model, ssp2-rcp scenario, and year. same order as when we created the ndb
ndb.write_db_to_brightway([
    dbName(0),
    dbName(1),
    dbName(2),
    dbName(3),
    dbName(4),
    dbName(5)
])

Write new database(s) to Brightway2.
Prepare database 1.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
24032 datasets
682007 exchanges
0 unlinked exchanges
  
Title: Writing activities to SQLite3 database:
  Started: 10/27/2022 12:37:27
  Finished: 10/27/2022 12:37:49
  Total time elapsed: 00:00:21
  CPU %: 94.20
  Memory %: 2.80
Created database: EI 3.8 cutoff image SSP2-Base 2050
Prepare database 2.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
24032 datasets
683138 exchanges
0 unlinked exchanges
  
Title: Writing activities to SQLite3 database:
  Started: 10/27/2022 12:39:54
  Finished: 10/27/2022 12:40:16
  Total time elapsed: 00:00:21
  CPU %: 94.20
  Memory %: 2.96
Created database: EI 3.8 cutoff image SSP2-RCP26 2050
Prepare database 3.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
24032 datasets
681352 exchanges
0 unlinked exchanges
  
Title: Writing activitie

In [39]:
bw.databases

Databases dictionary with 10 object(s):
	EI 3.8 cutoff image SSP2-Base 2050
	EI 3.8 cutoff image SSP2-RCP19 2050
	EI 3.8 cutoff image SSP2-RCP26 2050
	EI 3.8 cutoff remind SSP2-Base 2050
	EI 3.8 cutoff remind SSP2-PkBudg1150 2050
	EI 3.8 cutoff remind SSP2-PkBudg500 2050
	biosphere3
	ei 3.8 cutoff
	super_db_2022-10-26
	super_db_2022-10-27

# superstructure
when exporting the database as sueprstructure, we need a few mroe steps to share the superstructure database with the team (because of the scenario diff file)

In [67]:
ndb.write_superstructure_db_to_brightway()

UsageError: Cell magic `%%databases` not found.


In [108]:
#automatisation of saving altest super_db name
superdb_names = [key for key in bw.databases.keys() if key.startswith('super_db')]
superdb_names.sort()#make sure they are sorted
superdb=superdb_names[-1]#take the last one
print(superdb)

super_db_2022-10-27


In [106]:
#call from linux terminal to move the scenario diff file to our local github folder. The filename includes the date of generation. This is currently not automized, so remeber to change the filename according to the output from write_superstructure_db_to_brightway() 
!cp /opt/tljh/user/envs/bw/lib/python3.10/site-packages/premise/data/export/scenario\ diff\ files/scenario_diff_$superdb\.xlsx /home/jupyter-simb/moeschpond/

In [40]:
bw.backup_project_directory("moeschpond1")

Creating project backup archive - this could take a few minutes...


In [120]:
things=! ls ~/

! ls ~/

Autumn-School-2022
brightway2-project-moeschpond1-backup.26-October-2022-03-51PM.tar.gz
brightway2-project-moeschpond1-backup.26-October-2022-03-59PM.tar.gz
brightway2-project-moeschpond1-backup.27-October-2022-08-40AM.tar.gz
brightway2-project-moeschpond1-backup.27-October-2022-09-49AM.tar.gz
brightway2-project-moeschpond1-backup.27-October-2022-09-50AM.tar.gz
brightway2-project-moeschpond1-backup.27-October-2022-12-53PM.tar.gz
hybridization_data
moeschpond
pylcaio
scratch
Start-here.ipynb
teaching-material


In [121]:
#automatic update the string to the msot recent backup filename from above
backups = [key for key in things if key.startswith('brightway2-project-')]
backups.sort()#make sure they are sorted
backup=backups[-1]#take the last one
print(backup)

brightway2-project-moeschpond1-backup.27-October-2022-12-53PM.tar.gz


In [122]:
#DO NOT PUT ECOINVENT ON PUBLIC GITHUB
#copy the file to shared directory on server.
!cp ~/$backup /srv/scratch/u.2/

In [37]:
import brightway2 as bw
bw.restore_project_directory("/srv/scratch/u.2/"+backup)

Restoring project backup archive - this could take a few minutes...


'moeschpond1'

In [123]:
#check if db restored correctly
bw.databases

Databases dictionary with 10 object(s):
	EI 3.8 cutoff image SSP2-Base 2050
	EI 3.8 cutoff image SSP2-RCP19 2050
	EI 3.8 cutoff image SSP2-RCP26 2050
	EI 3.8 cutoff remind SSP2-Base 2050
	EI 3.8 cutoff remind SSP2-PkBudg1150 2050
	EI 3.8 cutoff remind SSP2-PkBudg500 2050
	biosphere3
	ei 3.8 cutoff
	super_db_2022-10-26
	super_db_2022-10-27